<a href="https://colab.research.google.com/github/tourfade/uqtr-selmet/blob/main/UsingDarioConf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This use  E[sben Jannik Bjerrum1,*, Mads Glahder1 and Thomas Skov](https://arxiv.org/pdf/1710.01927v1.pdf) that inspired [Dario](https://github.com/dario-passos/DeepLearning_for_VIS-NIR_Spectra)  configuration without bayesian optimization of hyper parameters.
Here is the completes code:
https://github.com/dario-passos/DeepLearning_for_VIS-NIR_Spectra/blob/master/notebooks/Bjerrum2017_CNN/BayesOpt_CNN1.2.ipynb

In [ ]:
!pip install livelossplot --quiet

In [1]:
!git clone https://github.com/EBjerrum/Deep-Chemometrics.git
import sys


Cloning into 'Deep-Chemometrics'...
remote: Enumerating objects: 43, done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 43
Unpacking objects: 100% (43/43), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

sys.path.insert(0,'/content/Deep-Chemometrics')
from livelossplot.tf_keras import PlotLossesCallback
from ChemUtils import GlobalStandardScaler, EmscScaler
import os, random
import io 
import pandas as pd
from scipy.io import arff
import tensorflow as tf
from tensorflow.keras import activations, optimizers
from tqdm.keras import TqdmCallback
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Dropout, Flatten, AveragePooling1D, MaxPooling1D, GaussianNoise, Reshape,BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import urllib.request
import matplotlib.pyplot as plt
from google.colab import files


In [4]:
opt = optimizers.Adam(learning_rate=0.0004)
#opt = optimizers.SGD( learning_rate=0.0001, momentum=0.01,  name="SGD")
#opt = optimizers.RMSprop(learning_rate=0.001)
epochs = 250
batch_size = 45
test_size = 0.2

In [12]:
 
 
def build_augmented_dataset(df):
    Y = df[df.columns[-1]] 
    #print("===================> Y:",Y.shape)
    X = df[df.columns[0:-1]]
    #print("===================> X:",X.shape)
    df1 = df.sample(1000)
    Y1 = df1[df1.columns[-1]]
    #print("===================> Y1:",Y1.shape)
    X1 = df1[df1.columns[0:-1]] 
    #print("===================> X1:",X1.shape)
    X1 = X1.mul(np.abs(10*np.random.rand(1000)), axis='index')
    

    X = X.append(X1)
    #print("===================> X:",X.shape)
    Y = Y.append(Y1)
    #print("===================> Y:",Y.shape)
    #X = X.values.reshape(X.shape[0], X.shape[1], 1)
    return (X, Y)

def build_emc_augmented(df):
    Y = df[df.columns[-1]] 
    #print("===================> Y:",Y.shape)
    X = df[df.columns[0:-1]]
    emsc = EmscScaler()
    emsc.fit(X)

    X_emsc0 = emsc.fit_transform(X.to_numpy())
    #x_train2_emsc0 = emsc.transform(x_train_2)
    #x_test1_emsc0 = emsc.transform(x_test_1)
    #x_test2_emsc0 = emsc.transform(x_test_2)


    ## Second: Rescale the EMSC data (more or less to near [-1, 1])
    X_scaler0 = GlobalStandardScaler(normfact=2) # normfact=2 ensure that we are dividing by 2. std 

    ## We use the avalaible data from the training set to create the scaler, and use that 
    ## same scaler to scale the other (unseen) subsets
    X_emsc = X_scaler0.fit_transform(X_emsc0)

    return (X_emsc, Y)

def build_dataset(df):
    Y = df[df.columns[-1]] 
    X = df[df.columns[0:-1]]
    X = X.values.reshape(X.shape[0], X.shape[1], 1)
    return (X, Y)

In [17]:
# Train function
#def root_mean_squared_error(y_true, y_pred):
 #       return K.sqrt(K.mean(K.square(y_pred - y_true))) 


   

def build_classifier(x_train, y_train):
    
    model = get_model()
    checkpoint_path = "train/selmet.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    model.compile(loss= tf.keras.losses.Huber,
                  optimizer = opt, 
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    # Create checkpoint callback
    rdlr = ReduceLROnPlateau(patience=25, factor=0.5, min_lr=1e-6, monitor='val_loss', verbose=1)
    early_stop = EarlyStopping(monitor='loss', min_delta=4e-4, patience=50, mode='auto', \
                                           restore_best_weights=True)
    plot_losses = PlotLossesCallback()
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                     save_weights_only = True,
                                                     verbose=0, 
                                                     callbacks=[early_stop])
    
    x_cal, x_val, y_cal, y_val = train_test_split(x_train, y_train, test_size=test_size, shuffle= True)

    
    result = model.fit(x_cal, y_cal, 
                        validation_data=(x_val, y_val),
                        callbacks = [cp_callback], 
                        #steps_per_epoch = 406,
                        batch_size = batch_size,
                        shuffle = True,
                        epochs = epochs )
    model.summary()
    # summarize history for accuracy
    plt.figure( )
    plt.subplots(figsize = (10, 5))
    plt.title('accuracies-loss')
    rmqe = result.history['root_mean_squared_error']
    plt.plot(rmqe, color = 'g', label='train')
    val_rmqe = result.history['val_root_mean_squared_error']
    plt.plot(val_rmqe, color = 'r', label='val')
    norm_loss = result.history['loss']/np.max(result.history['loss'])
    plt.plot(norm_loss, color = 'b', label='loss')
    plt.legend()
    plt.show()
    return checkpoint_path

In [7]:
def test_classifier(x_test, y_test, path):
    model = get_model()
    model.load_weights(path).expect_partial()
    model.compile(loss= tf.keras.losses.MSE,
                  optimizer=opt,
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
  
    
    print('(loss, rmse)', model.evaluate(x_test, y_test))
    return model
    

In [8]:
input_dim = 700
def get_model():
    model = Sequential()

    model.add(GaussianNoise(0.001, input_shape=(input_dim,)))
    model.add(Reshape((input_dim, 1)))
    
    model.add(Conv1D(filters=54, kernel_size=22, padding='same', activation=activations.relu))
    model.add(Conv1D(filters=42, kernel_size=29, padding='same', activation=activations.relu))
    model.add(Flatten()) 
    model.add(Dropout(0.38)) 
    model.add(Dense(289,  activation=activations.linear))
    model.add(Dense(1, activation=activations.linear)) 
    return model

In [13]:
train_df = pd.read_csv("/content/drive/MyDrive/cirad/selmet/adf/adf_train_data_p.csv", sep=',')
x_train, y_train = build_emc_augmented(train_df)



mean = x_train.mean()
std =  x_train.std()

x_train = (x_train - mean)/std

#x_train.tail(5).transpose().plot()


In [ ]:
path =  build_classifier(x_train, y_train) 


Epoch 1/250
161/161 [==============================] - 90s 553ms/step - loss: 135.4006 - root_mean_squared_error: 11.6362 - val_loss: 35.4389 - val_root_mean_squared_error: 5.9531
Epoch 2/250
161/161 [==============================] - 85s 526ms/step - loss: 36.5935 - root_mean_squared_error: 6.0493 - val_loss: 29.1672 - val_root_mean_squared_error: 5.4007
Epoch 3/250
161/161 [==============================] - 87s 540ms/step - loss: 32.4077 - root_mean_squared_error: 5.6928 - val_loss: 27.5494 - val_root_mean_squared_error: 5.2487
Epoch 4/250
161/161 [==============================] - 88s 548ms/step - loss: 30.0936 - root_mean_squared_error: 5.4858 - val_loss: 24.1879 - val_root_mean_squared_error: 4.9181
Epoch 5/250
161/161 [==============================] - 85s 525ms/step - loss: 29.0502 - root_mean_squared_error: 5.3898 - val_loss: 26.2880 - val_root_mean_squared_error: 5.1272
Epoch 6/250
161/161 [==============================] - 88s 549ms/step - loss: 26.5099 - root_mean_squared_er

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/cirad/selmet/adf/adf_test_data_p.csv", sep=',')

x_test, y_test = build_emc_augmented(test_df)
x_test =  (x_test - mean)/std


model = test_classifier(x_test, y_test, path)

y_predicted =  model.predict(x_test) 
y_g_truth =    y_test

plt.figure( )
plt.subplots(figsize = (10, 5))
plt.title('plot')
 

y_predicted = y_predicted.reshape(1,-1)[0]
plt.scatter(y_g_truth, y_predicted.transpose() ,  s=10, c ='r', label='pred')
plt.legend()
plt.show()


 